In [4]:
import mlflow
from lung_function.modules.compute_metrics import icc, metrics
from mlflow import log_params
import pandas as pd
from pathlib import Path


/home/jjia/.local/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/jjia/.local/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [1]:
# for new experiments
b_ls = []
b_ls.append({0:2521, 1:2522, 2:2531, 3:2544, 4:2545, 'name':''})
b_ls.append({0:2527, 1:2528, 2:2546, 3:2556, 4:2566, 'name':''})
b_ls.append({0:2529, 1:2530, 2:2547, 3:2557, 4:2567, 'name':''})
b_ls.append({0:2532, 1:2533, 2:2548, 3:2558, 4:2579, 'name':''})

b_ls.append({0:2534, 1:2535, 2:2553, 3:2563, 4:2588, 'name':''})
b_ls.append({0:2536, 1:2537, 2:2552, 3:2562, 4:2587, 'name':''})
b_ls.append({0:2538, 1:2539, 2:2551, 3:2561, 4:2586, 'name':''})
b_ls.append({0:2540, 1:2541, 2:2549, 3:2559, 4:2578, 'name':''})

b_ls.append({0:2542, 1:2543, 2:2550, 3:2560, 4:2581, 'name':''})
b_ls.append({0:2568, 1:2569, 2:2580, 3:2590, 4:2599, 'name':''})
b_ls.append({0:2570, 1:2571, 2:2582, 3:2589, 4:2597, 'name':''})
b_ls.append({0:2572, 1:2573, 2:2583, 3:2592, 4:2598, 'name':''})

b_ls.append({0:2574, 1:2575, 2:2585, 3:2593, 4:2602, 'name':''})
b_ls.append({0:2576, 1:2577, 2:2584, 3:2591, 4:2596, 'name':''})
b_ls.append({0:2594, 1:2595, 2:2604, 3:2612, 4:2616, 'name':''})
b_ls.append({0:2600, 1:2601, 2:2603, 3:2609, 4:2613, 'name':''})

b_ls.append({0:2605, 1:2606, 2:2610, 3:2614, 4:2617, 'name':''})
b_ls.append({0:2607, 1:2608, 2:2611, 3:2615, 4:2618, 'name':''})

b_ls.append({0:902, 1:903, 2:915, 3:921, 4:926, 'name':''})
b_ls.append({0:904, 1:905, 2:914, 3:919, 4:924, 'name':''})
b_ls.append({0:906, 1:907, 2:918, 3:925, 4:930, 'name':''})
b_ls.append({0:908, 1:909, 2:916, 3:923, 4:929, 'name':''})
b_ls.append({0:910, 1:911, 2:920, 3:927, 4:931, 'name':''})
b_ls.append({0:912, 1:913, 2:917, 3:922, 4:928, 'name':''})

b_ls.append({0:2258, 1:2259, 2:2269, 3:2271, 4:2272, 'name':''})
b_ls.append({0:2140, 1:2141, 2:2162, 3:2175, 4:2191, 'name':''})
b_ls.append({0:2138, 1:2139, 2:2160, 3:2173, 4:2190, 'name':''})
b_ls.append({0:2148, 1:2149, 2:2159, 3:2172, 4:2189, 'name':''})
b_ls.append({0:2144, 1:2145, 2:2158, 3:2171, 4:2185, 'name':''})
b_ls.append({0:2142, 1:2143, 2:2154, 3:2167, 4:2180, 'name':''})

b_ls.append({0:2194, 1:2195, 2:2249, 3:2262, 4:2270, 'name':''})
# b_ls.append({0:429, 1:430, 2:432, 3:434, 4:436, 'name':''})

b_ls.append({0:2667, 1:2668, 2:2679, 3:2690, 4:2701, 'name':''})
b_ls.append({0:2665, 1:2666, 2:2677, 3:2688, 4:2699, 'name':''})
b_ls.append({0:2663, 1:2664, 2:2678, 3:2689, 4:2700, 'name':''})
b_ls.append({0:2661, 1:2662, 2:2676, 3:2687, 4:2698, 'name':''})
b_ls.append({0:2655, 1:2658, 2:2675, 3:2686, 4:2697, 'name':''})
b_ls.append({0:2654, 1:2657, 2:2674, 3:2685, 4:2696, 'name':''})
b_ls.append({0:2651, 1:2652, 2:2672, 3:2683, 4:2691, 'name':''})
b_ls.append({0:2649, 1:2650, 2:2670, 3:2682, 4:2693, 'name':''})
b_ls.append({0:2647, 1:2648, 2:2669, 3:2680, 4:2692, 'name':''})
b_ls.append({0:2645, 1:2646, 2:2673, 3:2684, 4:2695, 'name':''})
b_ls.append({0:2643, 1:2644, 2:2671, 3:2681, 4:2694, 'name':''})


b_ls = [{0:2731, 1:2733, 2:2734, 3:2737, 4:2739, 'name':''}]
b_ls.append({0:2730, 1:2732, 2:2735, 3:2736, 4:2738, 'name':''})
b_ls.append({0:2714, 1:2715, 2:2717, 3:2722, 4:2725, 'name':''})
b_ls.append({0:2712, 1:2713, 2:2718, 3:2723, 4:2727, 'name':''})
b_ls.append({0:2710, 1:2711, 2:2716, 3:2719, 4:2724, 'name':''})

fold_ex_dt_ls = b_ls

## merging 4 folds

In [2]:
def ensemble_4folds_testing(fold_ex_dt_ls):
    parent_dir = '/home/jjia/data/lung_function/lung_function/scripts/results/experiments/'

    for fold_ex_dt in fold_ex_dt_ls:
        dir0 = parent_dir + str(fold_ex_dt[0])
        label_fpath = dir0  + '/test_label.csv'
        ave_fpath =dir0  + '/test_pred.csv'
        output_file_path = Path(ave_fpath)
        output_file_path.parent.mkdir(parents=True, exist_ok=True)
        
        df_ls = []
        for i in [1,2,3,4]:
            data_fpath = parent_dir + str(fold_ex_dt[i]) + '/test_pred.csv'
            df = pd.read_csv(data_fpath,index_col=0)
            df_ls.append(df)
        df_ave = (df_ls[0] + df_ls[1] + df_ls[2] + df_ls[3])/4
        df_ave.to_csv(ave_fpath)
        
        label_fpath_fold1 = parent_dir + str(fold_ex_dt[i]) + '/test_label.csv'
        df_label = pd.read_csv(label_fpath_fold1,index_col=0)
        df_label.to_csv(label_fpath)
        
        print(ave_fpath)

def ensemble_4folds_validation(fold_ex_dt_ls):
    parent_dir = '/home/jjia/data/lung_function/lung_function/scripts/results/experiments/'

    for fold_ex_dt in fold_ex_dt_ls:
        dir0 = parent_dir + str(fold_ex_dt[0])
        pred_all_fpath = dir0  + '/valid_pred.csv'
        label_all_fpath = dir0  + '/valid_label.csv'
        output_file_path = Path(pred_all_fpath)
        output_file_path.parent.mkdir(parents=True, exist_ok=True)
        
        df_pred_ls, df_label_ls = [], []
        for i in [1,2,3,4]:
            data_fpath = parent_dir + str(fold_ex_dt[i]) + '/valid_pred.csv'
            label_fpath = parent_dir + str(fold_ex_dt[i]) + '/valid_label.csv'
            df_pred = pd.read_csv(data_fpath,index_col=0)
            df_label = pd.read_csv(label_fpath,index_col=0)

            df_pred_ls.append(df_pred)
            df_label_ls.append(df_label)
        df_pred_valid = pd.concat(df_pred_ls)
        df_label_valid = pd.concat(df_label_ls)
        
        df_pred_valid.to_csv(pred_all_fpath)
        df_label_valid.to_csv(label_all_fpath)
        print(pred_all_fpath)

        
        
        

In [5]:
ensemble_4folds_testing(fold_ex_dt_ls)
ensemble_4folds_validation(fold_ex_dt_ls)

/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2731/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2730/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2714/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2712/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2710/test_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2731/valid_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2730/valid_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2714/valid_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2712/valid_pred.csv
/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2710/valid_pred.csv


## adding parameters

In [8]:
def mae(pred_fpath, label_fpath, ignore_1st_column=True):
    mae_dict = {}

    label = pd.read_csv(label_fpath)
    pred = pd.read_csv(pred_fpath)
    if ignore_1st_column:
        pred = pred.iloc[: , 1:]
        label = label.iloc[: , 1:]
    if 'ID' == label.columns[0]:
        del label["ID"]
    if 'ID' == pred.columns[0]:
        del pred["ID"]

    original_columns = label.columns

    # ori_columns = list(label.columns)

    for column in original_columns:
        mae_value = (pred[column] - label[column]).abs().mean()
        
        prefix = label_fpath.split("/")[-1].split("_")[0]
        mae_dict['mae_' + prefix + '_' + column] = mae_value
    return mae_dict

In [10]:
mlflow.set_tracking_uri("http://nodelogin02:5000")
experiment = mlflow.set_experiment("lung_fun_db15")

# Create a run under the default experiment (whose id is '0').
client = mlflow.MlflowClient()
parent_dir = '/home/jjia/data/lung_function/lung_function/scripts/results/experiments/'

for i in fold_ex_dt_ls:
    print(i)
    experiment_id = i[0]
    run_ls = client.search_runs(experiment_ids=[experiment.experiment_id], run_view_type=1, filter_string=f"params.id LIKE '%{experiment_id}%'")
    if len(run_ls)!=0:
        run_ = run_ls[-1]
        run_id = run_.info.run_id
        
        with mlflow.start_run(run_id=run_id):
            for mode in ['valid', 'test']:
                label_fpath = f"{parent_dir}{i[1]}/{mode}_label.csv"
                pred_fpath = f"{parent_dir}{i[0]}/{mode}_pred.csv"

                # add icc
                icc_value = icc(label_fpath, pred_fpath, ignore_1st_column=True)
                icc_value_ensemble = {'ensemble_' + k:v  for k, v in icc_value.items()}  # update keys
                log_params(icc_value_ensemble)

                # add r
                r_p_value = metrics(pred_fpath, label_fpath, ignore_1st_column=True)
                r_p_value_ensemble = {'ensemble_' + k:v  for k, v in r_p_value.items()}  # update keys
                log_params(r_p_value_ensemble)
                
                # add mae
                mae_dict = mae(pred_fpath, label_fpath, ignore_1st_column=True)
                mae_ensemble = {'ensemble_' + k:v for k, v in mae_dict.items()}
                log_params(mae_ensemble)     

{0: 2731, 1: 2733, 2: 2734, 3: 2737, 4: 2739, 'name': ''}


ValueError: Either missing values are present in data or data are unbalanced. Please remove them manually or use nan_policy='omit'.